In [209]:
import pandas as pd
from datetime import datetime, timedelta

In [210]:
df_load = [] #Temp list for combining DFs
df_wind = []
df_solar = []

YEARS = ['2022', '2023', '2024']

for year in YEARS:
    df_load.append(pd.read_excel("Native_Load_{}.xlsx".format(year), index_col=0))
    df_wind.append(pd.read_excel("Solar_Wind_{}.xlsx".format(year), sheet_name="Wind Data", index_col= "Time (Hour-Ending)"))
    df_solar.append(pd.read_excel("Solar_Wind_{}.xlsx".format(year), sheet_name="Solar Data", index_col= "Time (Hour-Ending)"))
    
#df init
df_load = pd.concat(df_load)
df_wind = pd.concat(df_wind)
df_solar = pd.concat(df_solar)


In [211]:
problematic_rows = df_load[df_load.index.str.contains("DST", na=False)]
print(problematic_rows) #Rows with DST tag (not parsable by pandas datetime)
df_load.index = df_load.index.str.replace(" DST", "", regex=False)

                             COAST         EAST        FWEST        NORTH  \
Hour Ending                                                                 
11/06/2022 02:00 DST   9331.958672  1104.183137  5094.569030   752.622044   
11/05/2023 02:00 DST  10108.434971  1127.380115  5671.605921   904.897362   
11/03/2024 02:00 DST  12392.327195  1366.024767  6506.165503  1280.902827   

                             NCENT        SOUTH        SCENT         WEST  \
Hour Ending                                                                 
11/06/2022 02:00 DST   8844.275206  2888.351883  5113.815243   938.180082   
11/05/2023 02:00 DST   8789.382953  2784.580564  5598.843114   952.353719   
11/03/2024 02:00 DST  11266.437516  3746.732143  7021.316310  1046.335622   

                             ERCOT  
Hour Ending                         
11/06/2022 02:00 DST  34067.955296  
11/05/2023 02:00 DST  35937.478720  
11/03/2024 02:00 DST  44626.241884  


In [212]:
df_load.index = df_load.index.map(str)
for index, row in df_load.iterrows():

    try:
        date, time = index.split(" ", 1) #Check nans or other non-parseables
    except:
        print(f"Index: {index}")
        print(f"Inde Type: {type(index)}")
        df_load = df_load.drop(index)
        continue

    if time == '24:00': #Some stupid ahh reason 24:00 instead of 00:00
        date = pd.to_datetime(date)
        new_date = date + timedelta(days=1)  #Converts DD 24:00 to DD+1 00:00

        df_load.loc[new_date] = df_load.loc[index] #Copy pasta
        df_load = df_load.drop(index) #Deletes old unusable index
    
df_load.index = pd.to_datetime(df_load.index.astype(str), format='mixed', errors='coerce') #allows mix HH:MM and HH:MM:SS


num_na = pd.isna(df_load.index).sum()
print(f"Rows Missing Values: {num_na}")
print(df_load.dtypes)


Index: nan
Inde Type: <class 'str'>
Rows Missing Values: 0
COAST    float64
EAST     float64
FWEST    float64
NORTH    float64
NCENT    float64
SOUTH    float64
SCENT    float64
WEST     float64
ERCOT    float64
dtype: object


In [213]:
df_solar.index = pd.to_datetime(df_solar.index)
num_na = pd.isna(df_solar.index).sum()
print(f"Rows Missing Values: {num_na}")
print(df_solar.dtypes)

Rows Missing Values: 0
Date                            datetime64[ns]
ERCOT.LOAD                             float64
ERCOT.PVGR.GEN                         float64
Total Solar Installed, MW                int64
Solar Output, % of Load                float64
Solar Output, % of Installed           float64
Solar 1-hr MW change                   float64
Solar 1-hr % change                    float64
Daytime Hour                            object
Ramping Daytime Hour                    object
dtype: object


In [214]:
df_wind.index = pd.to_datetime(df_wind.index)
num_na = pd.isna(df_wind.index).sum()
print(f"Rows Missing Values: {num_na}")
print(df_wind.dtypes)

Rows Missing Values: 0
Date                           datetime64[ns]
ERCOT.LOAD                            float64
ERCOT.WIND.GEN                        float64
Total Wind Installed, MW                int64
Wind Output, % of Load                float64
Wind Output, % of Installed           float64
Wind 1-hr MW change                   float64
Wind 1-hr % change                    float64
dtype: object
